In [1]:
import xarray as xr
import numpy as np
from glob import glob
import gc

### Description

##### In this script, we are aggregating Copernicus CGLS-LAI data over agricultural land at 300m horizontal resolution to ~1km spatial resolution. CCI annual land cover maps are used to mask cropland areas. In steps 4 and 5 the masked CGLS-LAI data is aggregated to GLASS-LAI (5km) using the same masking conditions, to harmonize both datasets.
##### You will therefore need CGLS-LAI data, CCI land cover maps and potentially GLASS-LAI

### STEP 1: Generate agricultural masks for 1km resolution

####  ---- Filter on 100% agricultural land gridcells, then coarsen to ~1km (1/120deg) spatial resolution

In [2]:
# load in all ESA CCI landcover files at 300m
basedir = f'/base/dir/'
files   = sorted(glob(f'{basedir}external/landcover_esa_cci_300m/*.nc'))
years     = np.arange(1999,2015)

# settings
# --- coarsen by factor 3 (300m ->900m)
lat_factor = 3
lon_factor = 3
# --- agricultural classes CCI
agr_cls    = [10,11,20] 

In [3]:
# this takes a while! increasing cores can speed up the process, otherwise use multiprocessing to run files in parallel.
for f in files:
    yr          = f.split('-')[-2]
    ds          = xr.open_dataset(f)
    esa_lc      = ds['lccs_class']
    # mask on agricultural classess
    ag_mask     = esa_lc.isin(agr_cls).astype(int)
    # coarsen to ~1 kilometer
    ag_sum      = ag_mask.coarsen(lat=lat_factor, lon=lon_factor, boundary='trim').sum()
    # all 9 pixels need to be agricultural: if sum ==9 set to 1, else 0
    ag_sum      = ag_sum.isin(9).astype(int)
    print(yr)

    ds.close()

    # Replace NaN with -9999
    ag_mask_filled = ag_sum.fillna(-9999).astype('int16')  # or int8 if preferred
    ag_mask_filled.name = 'agri_100pct'
    # Set encoding so -9999 is treated as _FillValue
    encoding = {
        'agri_100pct': {
            'dtype': 'int16',
            '_FillValue': -9999,
            'zlib': True  # optional compression
        }
    }
    
    # Save to NetCDF
    ag_mask_filled.to_netcdf(f'{basedir}processed/CGLS_LAI/esa_cci_1km/{yr}_masked_1km.nc', encoding=encoding)

    #  Cleanup
    del esa_lc, ag_mask,ag_sum, ag_mask_filled
    gc.collect()

### STEP 2: Apply nearest neighbour to match CCI cropmask (1/120 degrees) with CGLS (1/112 degrees) 

##### option A) run _resample_cgls.sh_ (in postprocessing_bash folder)

##### option B) load CDO in python and use following CDO commands:

In [4]:
def CDO_nearest_neighbour(years):
    grid_112.txt = cdo.griddes(input_CGLS.nc) 
    for y in years:
        fin =  f'esa_cci_1km/{y}_masked_1km.nc'
        fout = f'esa_cci_cgls_1km/{y}_cgls_1km.nc'
        cdo.remapnn(grid_112.txt, fin, fout)

##### --> we now have a new folder "_esa_cci_cgls_1km_" with cropmasks that matches the lat/lon values of CGLS LAI

### STEP 3: mask out LAI using the output from step 2

In [4]:
# select paths ESA_CCI mask and CGLS_LAI
path_lai  = f'{basedir}external/lai_cgls_1km/'
path_mask = f'{basedir}processed/CGLS_LAI/esa_cci_cgls_1km/'

##### we consider only areas that have agriculture during entire period: all years in mask need to be equal to 1

In [12]:
# Do this once. Again, requires many CPUs: can be done file-by-file instead
ds_mask_all  = xr.open_dataset(f'{path_mask}99_14_masked.nc') # merged years into 1 file with CDO

def mask_agri_allyears(mask_merged):
    ''' input is CGLS 3D mask in time/lat/lon dimension. Output is a 2D (lat/lon) mask '''
    ds_mask = (mask_merged['agri_100pct'] == 1).all(dim='time')
    ds_mask = ds_mask.astype(np.int8)
    #save output:
    ds_mask.to_netcdf(f'{basedir}processed/CGLS_LAI/esa_cci_cgls_1km/agri_all1.nc')
    
    return ds_mask

In [4]:
ds_mask = xr.open_dataset(f'{basedir}processed/CGLS_LAI/esa_cci_cgls_1km/agri_all1.nc')

# Replace mask fill values with NaN
ds_mask = ds_mask.where(ds_mask != -9999)
ds_mask = ds_mask.where(ds_mask != 0)

##### loop over years and save output

In [17]:
for year in years:
    print(year)
    fs_lai = sorted(glob(f'{path_lai}/*_{year}*.nc'))
    if len(fs_lai) != 36:
        print(f'warning: number of files for year {year} incorrect')
    for flai in fs_lai:
        ds_lai  = xr.open_dataset(flai)
        datestamp = flai.split('_')[5]
        
        # Reset mask dimensions to match ds_lai
        ds_mask['time'] = ds_lai['time']
        ds_mask['lat']  = ds_lai['lat']
        ds_mask['lon']  = ds_lai['lon']

        # multiply data with mask, filter out LAI for non-agricultural areas
        ds_lai['LAI_masked'] = ds_lai['LAI'] * ds_mask['agri_100pct']
        

        # Save to NetCDF
        print(datestamp)
        ds_lai['LAI_masked'].to_netcdf(f'{basedir}processed/CGLS_LAI/masked_nan/CGLS_{datestamp}_masked.nc')

        # Close dataset
        ds_lai.close()

1999
199901100000
199901200000
199901310000
199902100000
199902200000
199902280000
199903100000
199903200000
199903310000
199904100000
199904200000
199904300000
199905100000
199905200000
199905310000
199906100000
199906200000
199906300000
199907100000
199907200000
199907310000
199908100000
199908200000
199908310000
199909100000
199909200000
199909300000
199910100000
199910200000
199910310000
199911100000
199911200000
199911300000
199912100000
199912200000
199912310000
2000
200001100000
200001200000
200001310000
200002100000
200002200000
200002290000
200003100000
200003200000
200003310000
200004100000
200004200000
200004300000
200005100000
200005200000
200005310000
200006100000
200006200000
200006300000
200007100000
200007200000
200007310000
200008100000
200008200000
200008310000
200009100000
200009200000
200009300000
200010100000
200010200000
200010310000
200011100000
200011200000
200011300000
200012100000
200012200000
200012310000
2001
200101100000
200101200000
200101310000
2001021000

### Now we're going to bring in the second LAI dataset: GLASS at 0.05 degrees resolution

### STEP 4: mask GLASS LAI data

#### Use _ESA_CCI_cropmask_functions.ipynb_ to create masks from CCI data

In [ ]:
# datasets generated with functions from ESA_CCI_cropmask_functions
agri_mask   = xr.open_dataset(f'{basedir_glass}agriculture_mask_1992_2014.nc')
nonveg_mask = xr.open_dataset(f'{basedir_glass}nonveg_mask_1992_2014.nc')

In [ ]:
# CCI classes to consider
agri  = agri_mask['total'] 
urban = nonveg_mask['urban']
water = nonveg_mask['water']

In [ ]:
# Condition 1: agri ≥ 80 for all years
cond1 = (agri >= 80).all(dim='time')  # (lat, lon)

# Condition 2: agri ≥ 60 for all years AND agri + urban + water == 100 for all years
agri_ge_60 = (agri >= 60).all(dim='time')
sum_100 = ((agri + urban + water) == 100).all(dim='time')
cond2 = agri_ge_60 & sum_100

# Final mask: condition 1 OR condition 2
CCI_mask = xr.where(cond1 | cond2, 1.0, 0) 

In [ ]:
# loop over GLASS LAI to filter data
for glass_file in glass_files_sorted:
    print(glass_file)
    year       = glass_file.split('/')[-1][4:8]
    f          = xr.open_dataset(glass_file)
    fill_value = -9999.0

    # First align coordinates (lat, lon) to match the dataset
    CCI_mask = CCI_mask.assign_coords(lat=f['lat'], lon=f['lon'])
    
    # Then broadcast to (time, lat, lon)
    mask_expanded = CCI_mask.expand_dims(dim={'time': f['time']}, axis=0)

    # Apply mask
    lai_masked = f['LAI'] * mask_expanded

    lai_masked.name = "LAI"
    lai_masked.attrs = f['LAI'].attrs

    #make sure to keep original coordinates in new dataset
    masked_ds= xr.Dataset(
        {'LAI': lai_masked},
        coords={k: f[k] for k in f.coords}
    )

    masked_ds['lat'].attrs.update({'units':'degrees_north', 'standard_name':'latitude'})
    masked_ds['lon'].attrs.update({'units':'degrees_east', 'standard_name':'longitude'})

    encoding = {}
    for var in masked_ds.data_vars:
        dtype = masked_ds[var].dtype
        encoding[var] = {"_FillValue": np.float64(-9999.), "dtype": "float64"}
    # Copy global attributes
    masked_ds.attrs = f.attrs

    masked_ds.to_netcdf(f'{basedir_glass}/LAI_masked/GLASS_LAI_CCImasked_{year}.nc', encoding=encoding)

### STEP 5: match CGLS LAI with GLASS resolution / condition 

##### GLASS data is 0.05 degrees, which equals 6 times the 1/120 resolution of CGLS. To perform a fair comparison, we first aggregate CGLS to GLASS resolution

In [2]:
files = sorted(glob(f'{basedir}processed/CGLS_LAI/masked/CGLS*.nc'))

In [42]:
#f'{basedir}processed/CGLS_LAI/masked_5km/masked_from_GLASS/GLASS_2014_mask_nn.nc'
glass_mask = xr.open_dataset('glass_mask.nc')
glas_mask  = glass_mask.max(dim='time')

In [43]:
# mask (can also run in loop but saving time) need to define ds_coarse for first time to run this
ds_mask['mask'] = xr.where(ds_mask['LAI'] > 0, 1, np.nan) #(ds_mask > 0).astype(int)
ds_mask['lat'] = ds_coarse['lat'] 
ds_mask['lon'] = ds_coarse['lon']

In [ ]:
for f in files:
    timestamp = f.split('_')[2]
    print(timestamp)
    dir = f'{basedir}processed/CGLS_LAI/masked/'
    ds  = xr.open_dataset(f'{dir}CGLS_{timestamp}_masked.nc')

    lat_factor = 6
    lon_factor = 6    

    # aggregate new dataset at 0.05 degrees, by taking average of existing data
    ds_coarse     = xr.where(ds_mask['LAI'] > 0.0, ds_mask['LAI'], np.nan)
    ds_coarse     = ds.coarsen(lat=lat_factor, lon=lon_factor, boundary='trim').mean()
    ds_coarse     = ds_coarse * ds_mask['mask']

    # save output
    ds_coarse.to_netcdf(f'{basedir}processed/CGLS_LAI/masked_5km/CGLS_{timestamp}_masked.nc')

199901100000
199901200000
199901310000
199902100000
199902200000
199902280000
199903100000
199903200000
199903310000
199904100000
199904200000
199904300000
199905100000
199905200000
199905310000
199906100000
199906200000
199906300000
199907100000
199907200000
199907310000
199908100000
199908200000
199908310000
199909100000
199909200000
199909300000
199910100000
199910200000
199910310000
199911100000
199911200000
199911300000
199912100000
199912200000
199912310000
200001100000
200001200000
200001310000
200002100000
200002200000
200002290000
200003100000
200003200000
200003310000
200004100000
200004200000
200004300000
200005100000
200005200000
200005310000
200006100000
200006200000
200006300000
200007100000
200007200000
200007310000
200008100000
200008200000
200008310000
200009100000
200009200000
200009300000
200010100000
200010200000
200010310000
200011100000
200011200000
200011300000
200012100000
200012200000
200012310000
200101100000
200101200000
200101310000
200102100000
200102200000